# Notebook Title: AfyaMum Bot: AI-powered pregnancy support chatbot

## Introduction
This notebook utilizes the power of GPT-3.5 model by OpenAI, in combination with a comprehensive document on Maternal Reproductive Health to offer expectant mothers tailored information and support. Maternal health during pregnancy is of utmost importance, and this notebook aims to empower expectant mothers with personalized insights, risk assessment, prevention methods, and guidance for a healthier pregnancy journey.

## How it works?
1. **Data Input**: Expectant mothers provide information on how they feel through SMS

2. **Personalization**: Llama 2 will process the input data to generate personalized recommendations and assessments.

The Personalized Maternal Reproductive Health Guide, powered by Llama 2, aims to provide expectant mothers with the knowledge, tools, and support needed for a healthy and informed pregnancy journey.


In [6]:
#!pip install python-dotenv
#!pip install openai

In [12]:
#!pip install --upgrade langchain

In [14]:
#!pip install chromadb

## 1.0: Loading the required packages

In [7]:
import os
import openai
import sys
sys.path.append('../..')
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

In [8]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [9]:
langchain_tracing_v2 = os.environ['LANGCHAIN_TRACING_V2']
langchain_endpoint = os.environ['LANGCHAIN_ENDPOINT']
langchain_api_key = os.environ['LANGCHAIN_API_KEY']

In [16]:
# !pip install chromadb

In [15]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [16]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("Hello!")

'Hello! How can I assist you today?'

In [19]:
# !pip install tiktoken

In [20]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "Thanks for asking and I'm optimistic everything will be well!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "What are the major causes of pregnancy complications?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

"The major causes of pregnancy complications can include pre-existing health conditions, such as diabetes or high blood pressure, infections during pregnancy, and problems with the placenta or umbilical cord. However, it is important to note that each pregnancy is unique and complications can vary. Thanks for asking and I'm optimistic everything will be well!"

## Memory

In [21]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## ConversationalRetrievalChain

In [22]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [23]:
question = "Is infections during a pregnancy a major cause of pregnancy complications and what number of expectant women encounter these complications?"
result = qa({"question": question})

In [24]:
result['answer']

'Infections during pregnancy can indeed be a major cause of pregnancy complications. Certain infections, such as urinary tract infections, bacterial vaginosis, and sexually transmitted infections, can increase the risk of preterm labor, premature rupture of membranes, and low birth weight. Other infections, like rubella, cytomegalovirus, and Zika virus, can lead to birth defects in the baby.\n\nAs for the number of expectant women who encounter these complications, it can vary depending on various factors such as geographic location, socioeconomic status, access to healthcare, and individual health conditions. It is challenging to provide an exact number without specific data or statistics. It is always recommended for pregnant women to seek regular prenatal care and follow the guidance of healthcare professionals to minimize the risk of infections and associated complications.'

In [25]:
question = "Which infections among the following: urinary tract infections, bacterial vaginosis, and sexually transmitted infections is the major cause of complications in pregnancy?"
result = qa({"question": question})

In [26]:
result['answer']

'Among the infections listed, sexually transmitted infections (STIs) are generally considered to be the major cause of complications in pregnancy. STIs such as chlamydia, gonorrhea, syphilis, and HIV can pose significant risks to both the mother and the developing fetus if left untreated. It is important for pregnant individuals to receive regular prenatal care and be screened for STIs to prevent complications.'

## Create AfyaMum Bot

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [30]:
# !pip install pypdf

In [31]:
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 

In [33]:
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query = param.String("")
    db_response = param.List([])

    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        self.loaded_file = "./Data/Gynecology.pdf"
        self.qa = load_db(self.loaded_file, "stuff", 4)

    def convchain(self, query):
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        return self.answer

##  AfyaMum Bot Instance

In [35]:
# cb = cbfs()
# cb.convchain("What condition could I be experiencing with the following symptoms: back pains and swollen feet?")